# Código del script local para limpiar los datos desde archivos .csv

Este es el script principal que sirve para hacer la limpieza y formateo de datos desde los archivos .csv. Este archivo tiene que estar en una carpeta local junto con los .csv que se quieran procesar (y que estén en el formato entregado con el enunciado). Se recomienda leer documentación en GitHub antes.

En el siguiente bloque se importan las librerías necesarias para el código:
- pandas se usa para ordenar los datos en tablas
- glob se usa para acceder los archivos dentro de la carpeta local
- os se usa para usar varias funciones de modificación de archivos
- pyproj se usa para realizar transformaciones geoespaciales

In [1]:
import pandas as pd
import glob
import os
from pyproj import Transformer

En el siguiente bloque se setean variables auxiliares:

In [2]:
original_dir = "original"
upload_dir = "upload"
daily_dataframes = []

En el siguiente bloque se crean sub carpetas con las que se trabaja en el script. La carpeta "original" se usa para guardar los .csv originales que ya fueron procesados y "upload" para dejar los .csv listos para subir a la nube:

In [3]:
try:
    if os.path.isdir(original_dir):
        pass
    else:
        os.mkdir(original_dir)

    if os.path.isdir(upload_dir):
        pass
    else:
        os.mkdir(upload_dir)
except:
    print('Problema con la creación de directorios')

En el siguiente bloque se encuentra la lógica principal del script. Dentro del primer ciclo se revisan todos los archivos .csv dentro de la carpeta, se verifica el formato de estos archivos, se crea un dataframe por cada archivo de entrada y se inicializan variables auxiliares importantes. Estas variable importantes son:

- Diccionario con el código EPSG de la zona de la región junto a la ubicación del punto de origen de la grilla propuesta
- Diccionario con los valores que se usarán en el eje Y para transformar luego a strings

Una de las desventajas de esta solución temporalmente es la de tener que modificar estos diccionarios manualmente cuando se agreguen nuevas regiones y/o las strings no den a basto para el tamaño de la grilla.

Dentro del segundo ciclo se encuentran el proceso de limpieza y formateo de datos en sí. Para cada ciudad en los archivos .csv se pasa por este ciclo y se crea un dataframe.


In [4]:
# Se revisan todos los archivos .csv de la carpeta
for csv_file in glob.glob('*.csv'):

    # Para cada archivo:

    # Se lee el archivo .csv
    with open(csv_file, 'r') as f:
        trips = f.read()

        # Se verifica que los headers sean los correspondientes al formato
        first_line = trips.split('\n', 1)[0]
        if first_line != 'region,origin_coord,destination_coord,datetime,datasource':
            raise Exception('El archivo ' + csv_file + ' no está en el formato adecuado')

    # Se reemplazan algunos carácteres innecesarios del archivo
    replacements = [('POINT (', ''),
                    (')', ''),
                    (' ', ',')]
    for find, replacement in replacements:
        trips = trips.replace(find, replacement)

    # Se copia el texto limpio a un nuevo archivo con prefijo 'new_'
    new_trips_path = 'new_' + csv_file
    with open(new_trips_path, 'w') as f:
        f.write(trips)

    # Se inicializan variables auxiliares para la grilla
    cities = {'Turin': {'epsg': 32632, 'x0': 381000, 'y0': 4980000},
              'Hamburg': {'epsg': 32632, 'x0': 551000, 'y0': 5918000},
              'Prague': {'epsg': 32633, 'x0': 450000, 'y0': 5536000}}
    alphabet = {'1': 'a', '2': 'b', '3': 'c', '4': 'd', '5': 'e', '6': 'f', '7': 'g', '8': 'h', '9': 'i', '10': 'j',
                '11': 'k', '12': 'l', '13': 'm', '14': 'n', '15': 'o', '16': 'p', '17': 'q', '18': 'r', '19': 's',
                '20': 't', '21': 'u', '22': 'v', '23': 'w', '24': 'x', '25': 'y', '26': 'z', '27': 'aa', '28': 'ab',
                '29': 'ac', '30': 'ad', '31': 'ae', '32': 'af'}
    df_list = []

    # Se crea un dataframe de pandas con los datos del archivo
    new_trips_df = pd.read_csv(new_trips_path, header=0, names=['region', 'origin_x', 'origin_y', 'destination_x',
                                                                'destination_y', 'date', 'timestamp', 'datasource'])
    # Por cada ciudad en la lista "cities":
    for city in cities.keys():

        # Se crea un dataframe con las filas que coinciden con el nombre de la ciudad
        df = pd.DataFrame(new_trips_df[new_trips_df['region'] == city])

        # Se crean columnas con el tiempo en formato yyyy-mm-dd hh:mm:ss y además otra solo con la hora
        df['hora'] = df['timestamp'].str.split(':', 1).str[0]
        df['timestamp'] = df['date'] + ' ' + df['timestamp'] + ':00'
        df['timestamp'] = pd.to_datetime(df.timestamp)

        # Se crea un objeto Transformer, que permite transformar las coordenadas geográficas (Código EPSG: 4326) a
        # coordenadas proyectadas. Esta transformación depende de la zona del mundo, por esto se necesita el código
        # EPSG específico. Esta transformación se hace dado que las coordenadas proyectadas son más intuitivas de usar.
        transformer = Transformer.from_crs(4326, cities[city]['epsg'], always_xy=True)

        # Se transforman las coordenadas de ORIGEN y se asignan a columnas
        xx, yy = transformer.transform(df['origin_x'].values, df['origin_y'].values)
        df['easting_origin'] = xx
        df['northing_origin'] = yy

        # Se asignan valores enteros a las celdas de origen de la grilla. Para el eje Y estas se transforman a letras
        df['origin_cell_x'] = (df['easting_origin'] - cities[city]['x0']) / 1000 + 1
        df['origin_cell_x'] = df['origin_cell_x'].astype(int).astype(str)
        df['origin_cell_y'] = (df['northing_origin'] - cities[city]['y0']) / 1000 + 1
        df['origin_cell_y'] = df['origin_cell_y'].astype(int).astype(str)
        df.replace({'origin_cell_y': alphabet}, inplace=True)

        # El valor de la celda de origen corresponde a una string compuesta por el valor del entero de X y de la string
        # en Y
        df['origin_cell'] = df['origin_cell_x'] + df['origin_cell_y']

        # Se transforman las coordenadas de DESTINO y se asignan a columnas
        xx, yy = transformer.transform(df['destination_x'].values, df['destination_y'].values)
        df['easting_destination'] = xx
        df['northing_destination'] = yy

        # Se asignan valores enteros a las celdas de destino de la grilla. Para el eje Y estas se transforman a letras
        df['destination_cell_x'] = (df['easting_destination'] - cities[city]['x0']) / 1000 + 1
        df['destination_cell_x'] = df['destination_cell_x'].astype(int).astype(str)
        df['destination_cell_y'] = (df['northing_destination'] - cities[city]['y0']) / 1000 + 1
        df['destination_cell_y'] = df['destination_cell_y'].astype(int).astype(str)
        df.replace({'destination_cell_y': alphabet}, inplace=True)

        # El valor de la celda de destino corresponde a una string compuesta por el valor del entero de X y de la string
        # en Y
        df['destination_cell'] = df['destination_cell_x'] + df['destination_cell_y']

        # El dataframe resultante de la ciudad se agrega a una lista
        df_list.append(df)

    # Se concatena la lista de dataframes para dejar uno condensado con todas las ciudades
    df_cities = pd.concat(df_list)

    # Se agrega a la lista daily_dataframes el dataframe df_cities, que corresponde al dataframe creado desde UN .csv
    # Se toman las columnas con información significativa
    daily_dataframes.append(df_cities[['region', 'timestamp', 'hora', 'origin_cell', 'destination_cell', 'origin_x',
                                       'origin_y', 'destination_x', 'destination_y', 'datasource']])

    # Se elimina el archivo .csv temporal limpio creado al principio del ciclo. El original se mueve a una sub carpeta
    try:
        os.remove(new_trips_path)
        os.rename(csv_file, original_dir + '/' + csv_file)
    except:
        print('Error en el movimiento del archivos')

En este último bloque se crea el dataframe con todos los datos del día y se crea un .csv con esta información, que luego se mueve a una sub carpeta.

In [ ]:
# Se concatenan los dataframes de todos los archivos del día y se transforman en un .csv. Este archivo se mueve a una
# sub carpeta para cargarlo posteriormente.
if not daily_dataframes:
    print('No hay dataframes')
else:
    df_upload = pd.concat(daily_dataframes)
    df_upload.to_csv(upload_dir + '/formatted_trips.csv', index=False)